In [9]:
import cirq
from mitiq import MeasurementResult, Observable, PauliString, raw,zne
from mitiq.observable.observable import Observable
from mitiq.observable.pauli import PauliString

# problem setup

In [2]:
circuit = cirq.Circuit()
qubits = cirq.LineQubit.range(3)
circuit.append([
    cirq.H(qubits[2]),
    cirq.CX(qubits[1], qubits[2]),
    cirq.T(qubits[2])**-1,
    cirq.CX(qubits[0], qubits[2]),
    cirq.T(qubits[2]),
    cirq.CX(qubits[1], qubits[2]),
    cirq.T(qubits[2])**-1,
    cirq.CX(qubits[0], qubits[2]),
    cirq.T(qubits[1]),
    cirq.T(qubits[2]),
    cirq.CX(qubits[0], qubits[1]),
    cirq.H(qubits[2]),
    cirq.T(qubits[0]),
    cirq.T(qubits[1])**-1,
    cirq.CX(qubits[0], qubits[1]),
])
circuit.append([cirq.measure(qubit, key=f'q{index}') for index, qubit in enumerate(qubits)])

observable = Observable(PauliString("ZII"),PauliString("IZI"),PauliString("IIZ"))

print(circuit)

0: ──────────────────@──────────────────@───@───T──────@─────────M('q0')───
                     │                  │   │          │
1: ───────@──────────┼───────@───T──────┼───X───T^-1───X─────────M('q1')───
          │          │       │          │
2: ───H───X───T^-1───X───T───X───T^-1───X───T───H──────M('q2')─────────────


# make executer

In [3]:
from functools import partial
import numpy as np
from cirq.experiments.single_qubit_readout_calibration_test import (
    NoisySingleQubitReadoutSampler,
)
from mitiq import MeasurementResult

def noisy_readout_executor(circuit: cirq.Circuit,noise_level : 0.02, p0, p1, shots=8192) -> MeasurementResult:
    measurement = circuit[-1]
    circuit = circuit[:-1]

    circuit = circuit.with_noise(cirq.depolarize(noise_level))
    circuit.append(measurement)

    simulator = NoisySingleQubitReadoutSampler(p0=p0, p1=p1)
    result = simulator.run(circuit, repetitions=shots)
    bitstrings = np.column_stack([result.measurements[f'q{i}'] for i in range(len(qubits))])
    return MeasurementResult(bitstrings, qubit_indices=range(len(qubits)))


# evaluate expectation value

In [5]:
from mitiq.raw import execute as raw_execute

prob_flip = 0.05
noisy_executor = partial(noisy_readout_executor,noise_level = 0.02,p0=prob_flip,p1=prob_flip)
noisy_value = raw_execute(circuit,noisy_executor,observable)

ideal_executor = partial(noisy_readout_executor,noise_level = 0, p0=0,p1=0)
ideal_value = raw_execute(circuit,ideal_executor,observable)


print(f"ideal : {ideal_value:.3}")
print(f"error without mitigation : {noisy_value:.3}")

ideal : (3+0j)
error without mitigation : (1.95+0j)


# apply REM

In [7]:
from mitiq.rem import generate_inverse_confusion_matrix
from mitiq import rem

inverse_confusion_matrix = generate_inverse_confusion_matrix(3,prob_flip,prob_flip)

mitigated_result = rem.execute_with_rem(circuit,noisy_executor,observable,inverse_confusion_matrix=inverse_confusion_matrix)


print(f"Error with mitigation (REM): {mitigated_result:.3}")

Error with mitigation (REM): (2.14+0j)


# apply ZNE

In [ ]:
from mitiq.zne.inference import LinearFactory
from mitiq.zne.inference import RichardsonFactory
from mitiq.zne.scaling import fold_global

linear_fact = LinearFactory(scale_factors=[1.0,2.0,3.0])
Ricahrdson_fact = RichardsonFactory(scale_factors=[1.0,2.0,3.0])

linear_result = zne.execute_with_zne(circuit,noisy_readout_executor,factory=linear_fact,scale_noise=fold_global)
Richardson_result = zne.execute_with_zne(circuit,noisy_readout_executor,factory=Ricahrdson_fact,scale_noise=fold_global)

print(f"linear_fact: {linear_result :.{3}}")
print(f"Richardson_fact : {Richardson_result :.{3}}")
